In [2]:
import pandas as pd
import numpy as np
%pip install openpyxl -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [4]:
df = pd.read_csv('data/Train_rev1.csv')

In [5]:
df.shape

(244768, 12)

In [6]:
df.head(3)

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Id                  244768 non-null  int64 
 1   Title               244767 non-null  object
 2   FullDescription     244768 non-null  object
 3   LocationRaw         244768 non-null  object
 4   LocationNormalized  244768 non-null  object
 5   ContractType        65442 non-null   object
 6   ContractTime        180863 non-null  object
 7   Company             212338 non-null  object
 8   Category            244768 non-null  object
 9   SalaryRaw           244768 non-null  object
 10  SalaryNormalized    244768 non-null  int64 
 11  SourceName          244767 non-null  object
dtypes: int64(2), object(10)
memory usage: 22.4+ MB


In [8]:
df.describe(include='all')

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
count,2.447680e+05,244767,244768,244768,244768,65442,180863,212338,244768,244768,244768.000000,244767
unique,NaN,135435,242138,20986,2732,2,2,20812,29,97286,NaN,167
top,NaN,Business Development Manager,What is expected of you as a Registered Nurse ...,London,UK,full_time,permanent,UKStaffsearch,IT Jobs,"50,000-74,999 yearly",NaN,totaljobs.com
freq,NaN,921,18,15605,41093,57538,151521,4997,38483,1923,NaN,48149
mean,6.970142e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34122.577576,NaN
std,3.129813e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17640.543124,NaN
min,1.261263e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5000.000000,NaN
25%,6.869550e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21500.000000,NaN
50%,6.993700e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.000000,NaN
75%,7.162606e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42500.000000,NaN


In [9]:
df.shape

(244768, 12)

# 1. Drop columns

In [10]:
df.drop(columns=['Id', 'SalaryRaw', 'LocationRaw'], inplace=True)

# 2. Fill missing values

In [11]:
print('Missing values:')
df.isna().sum()/df.shape[0]*100

Missing values:


Title                  0.000409
FullDescription        0.000000
LocationNormalized     0.000000
ContractType          73.263662
ContractTime          26.108397
Company               13.249281
Category               0.000000
SalaryNormalized       0.000000
SourceName             0.000409
dtype: float64

In [12]:
def fill_missing(df):
    for col in df.columns:
        if df[col].dtype == 'O':  # object/string
            df = df.fillna(df[col].mode()[0])
        else:  # numbers
            df = df.fillna(df[col].mean())
    return df

In [13]:
df = fill_missing(df)

In [14]:
df.isna().sum()/df.shape[0]*100

Title                 0.0
FullDescription       0.0
LocationNormalized    0.0
ContractType          0.0
ContractTime          0.0
Company               0.0
Category              0.0
SalaryNormalized      0.0
SourceName            0.0
dtype: float64

# 3. Duplicates

In [15]:
df.duplicated().sum()

1

# 4. Geostandarization - web scrapping

In [15]:
%pip install requests -q
%pip install beautifulsoup4 -q
import requests
from bs4 import BeautifulSoup as bs
import re
import time
import random
import json

In [ ]:
def extract_number_from_text(text):
    match = re.search(r'(\d{1,3}(?:,\d{3})*)', text)
    return int(match.group(1).replace(',', '')) if match else None

In [ ]:
def select_population_from_table(table):
    for header_row in table.select('tr:has(th)'):
        th = header_row.select_one('th')
        if 'Population' not in th.get_text():
            continue

        # next <tr> sibling (population data may be here)
        next_row = header_row.find_next_sibling('tr')


        # 1. population in the same row
        td = header_row.select_one('td')
        if td:
            val = extract_number_from_text(td.get_text())
            if val:
                return val
            
        # 2. population in the next row <td>
        if next_row and next_row.select_one('td'):
            val = extract_number_from_text(next_row.select_one('td').get_text())
            if val:
                return val

        # 3. multiple population years (bulleted list)
        if next_row and re.match(r'\s*•\s*\d{4}', next_row.get_text()):
            # select all following <tr> until a break
            for tr in header_row.find_all_next('tr'):
                val = extract_number_from_text(tr.get_text())
                if val:
                    last_val = val
            return last_val
    return None

In [ ]:
def get_page(url, headers, retries=3, delay_range=(1, 3)):
    for _ in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                return response
            if response.status_code == 404:
                return None
        except requests.exceptions.RequestException:
            pass
        time.sleep(random.uniform(*delay_range))
    return None

In [ ]:
def fetch_infobox_table(url, headers, class_name='infobox'):
    page = get_page(url, headers)
    if not page:
        return None
    soup = bs(page.content, 'html.parser')
    return soup.find('table', class_=class_name)

In [ ]:
def get_population_for_city(city, headers):
    urls = [
        f'https://en.wikipedia.org/wiki/{city}',
        f'https://en.wikipedia.org/wiki/{city}_(county)'
    ]
    for url in urls:
        table = fetch_infobox_table(url, headers)
        if table:
            pop = select_population_from_table(table)
            if pop:
                return pop
    return None

In [ ]:
def get_population_for_location():
    headers = {"User-Agent": "LocationWebScrapper"}

    for city in cities:
        population = population_cache.get(city)
        if population is None:
            population = get_population_for_city(city, headers)
            if population:
                population_cache[city] = population
            else:
                not_working.add(city)
        print(f"{city}: {population}")

In [ ]:
# not_working = set()
# population_cache = {}

In [ ]:
cities = df['LocationNormalized'].unique().tolist()

In [ ]:
with open('data/population_cache.json', 'r', encoding='utf-8') as f:
    population_cache = json.load(f)

In [ ]:
not_working = [city for city in cities if city not in population_cache]

In [ ]:
print('not working: ', len(not_working))
print('cache: ', len(population_cache))
print("percent of coveraged population of cities: ", round((len(population_cache) / len(cities) * 100), 2), '%')

In [ ]:
not_working[:10]

In [ ]:
get_population_for_location()

In [408]:
with open('data/population_cache.json', 'w', encoding='utf-8') as f:
    json.dump(population_cache, f, ensure_ascii=False, indent=4)

# 4. Geostandarization

## 4.1. Get population data from geonames dataset

In [15]:
# selecting data only for GB - turn on once (long)
# cols = [
#     'geonameid','name','asciiname','alternatenames','lat','lon',
#     'feature_class','feature_code','country_code','cc2','admin1',
#     'admin2','admin3','admin4','population','elevation','dem','tz','moddate'
# ]

# geonames = pd.read_csv(
#     "allCountries.txt",
#     sep="\t",
#     names=cols,
#     usecols=['asciiname', 'alternatenames', 'country_code', 'feature_code', 'feature_class', 'admin1', 'admin2', 'admin3', 'lon', 'lat', 'population'],
#     dtype=str,
#     header=None
# )

# geonames_gb = geonames[geonames['country_code'] == 'GB'].copy().reset_index(drop=True)
# geonames_gb = geonames_gb[geonames_gb['feature_class'].isin(['P', 'A'])].reset_index()
# geonames_gb.loc[geonames_gb['feature_code'] == 'PCLI', 'asciiname'] = 'UK'
# geonames_gb.to_csv('geonames_gb.csv')

In [34]:
geonames_gb = pd.read_csv('geo_datasets/geonames_gb.csv')
geonames_gb.rename(columns={'asciiname': 'name'}, inplace=True)

## 4.2. Get population for all locations where it is directly possible

In [35]:
# get population for locations
pop_dict = geonames_gb['population'].copy()
pop_dict = geonames_gb.set_index(geonames_gb['name'].str.lower().str.strip())['population'].to_dict()

df['LocationPopulation'] = df['LocationNormalized'].str.lower().str.strip().map(lambda x: pop_dict.get(x))

In [36]:
def print_missing_info():
    print(f"Missing data in population of location: {round(df[df['LocationPopulation'].isna()]['LocationNormalized'].count() / len(df) * 100, 2)}%, {df[df['LocationPopulation'].isna()]['LocationNormalized'].count()} cases")
    print()
    print(df[df['LocationPopulation'].isna()]['LocationNormalized'].value_counts()[:5])

In [37]:
print_missing_info()

Missing data in population of location: 12.44%, 30460 cases

LocationNormalized
South East London    11713
Central London        2607
West Midlands         2540
Berkshire             1502
West Yorkshire        1072
Name: count, dtype: int64


## 4.3. Remove directions and assign population to other fitting names

In [38]:
directions = ['North', 'South', 'East', 'West', 'Central']
df['LocationNormalized'] = df['LocationNormalized'].replace(
    directions, '', regex=True
).str.strip()

missing_mask = df['LocationPopulation'].isna()
missing_locations = df.loc[missing_mask, 'LocationNormalized'].str.lower().str.strip()

pop_dict_missing = {loc: pop_dict.get(loc, np.nan) for loc in missing_locations}

df.loc[missing_mask, 'LocationPopulation'] = missing_locations.map(pop_dict_missing)

In [39]:
print_missing_info()

Missing data in population of location: 4.58%, 11216 cases

LocationNormalized
Midlands                    3456
Berkshire                   1502
Cheshire                     871
Yorkshire and Humberside     683
Bedfordshire                 544
Name: count, dtype: int64


## 4.4. Find population for Midlands in NUT regions

In [40]:
# remove locations out of GB
uk_lat_mask = (geonames_gb['lat'] >= 49) & (geonames_gb['lat'] <= 61)
uk_lon_mask = (geonames_gb['lon'] >= -10) & (geonames_gb['lon'] <= 2)
geonames_gb = geonames_gb[(geonames_gb['country_code'] == 'GB') & (uk_lat_mask) & (uk_lon_mask)]

In [41]:
nuts = pd.read_excel("geo_datasets/NUTS.xlsx")
nuts['NUTS118NM'] = nuts['NUTS118NM'].str.replace('(England)', '', regex=False).str.strip()
nuts = nuts.rename(columns={'NUTS118NM': 'name', 'LONG': 'lon', 'LAT': 'lat'})

In [42]:
# find the closest point in geonames in nuts
from scipy.spatial import cKDTree
tree = cKDTree(geonames_gb[['lat', 'lon']].values)
nuts_coords = nuts[['lat', 'lon']].values

distances, indices = tree.query(nuts_coords, k=1)  # k=1 -> 1 neighbour

nuts['population'] = geonames_gb.iloc[indices]['population'].values
nuts_population = dict(zip(nuts['name'], nuts['population']))

In [43]:
from typing import Counter

# combine West and East Midlands
nuts_population = {**{k: v for k, v in nuts_population.items() if 'Midlands' not in k},
                 **{'Midlands': sum(v for k, v in nuts_population.items() if 'Midlands' in k)}}

In [44]:
nuts_population

{'North East': 1126,
 'North West': 0,
 'Yorkshire and The Humber': 0,
 'East of England': 686,
 'London': 10750,
 'South East': 0,
 'South West': 0,
 'Wales': 0,
 'Scotland': 8830,
 'Northern Ireland': 0,
 'Midlands': 50878}

In [45]:
# impute nuts locations
population_from_dict = df['LocationNormalized'].map(nuts_population)

mask = ((df['LocationPopulation'].isnull()) | (df['LocationPopulation'] == 0)) & population_from_dict.notnull()

df.loc[mask, 'LocationPopulation'] = population_from_dict[mask]

In [46]:
print_missing_info()

Missing data in population of location: 3.17%, 7760 cases

LocationNormalized
Berkshire                   1502
Cheshire                     871
Yorkshire and Humberside     683
Bedfordshire                 544
Edinburgh Technopole         408
Name: count, dtype: int64


## 4.5. Cast rest of cases as 'UK'

In [47]:
mask = df['LocationPopulation'].isna() | (df['LocationPopulation'] == 0)
uk_pop = df.loc[df['LocationNormalized'].str.lower().eq('uk'), 'LocationPopulation'].dropna().iloc[0] if any(df['LocationNormalized'].str.lower().eq('uk')) else np.nan
df.loc[mask, ['LocationNormalized', 'LocationPopulation']] = ['UK', uk_pop]

In [48]:
print_missing_info()

Missing data in population of location: 0.0%, 0 cases

Series([], Name: count, dtype: int64)


In [49]:
df['LocationPopulation'].value_counts().head()

LocationPopulation
66488991.0    109313
8961989.0      45511
541263.0        3516
50878.0         3456
1157603.0       3061
Name: count, dtype: int64

In [50]:
df.drop(columns=['LocationNormalized'], inplace=True)

# Word2Vec

In [52]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
%pip install gensim -q
from gensim.models import Word2Vec
import multiprocessing

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Note: you may need to restart the kernel to use updated packages.


In [55]:
# tokenization
def tokenize_text(text):
    return word_tokenize(text.lower())

In [ ]:
titles = df['Title'].apply(tokenize_text).tolist()
descriptions = df['FullDescription'].apply(tokenize_text).tolist()

In [ ]:
all_sentences = titles + descriptions

In [ ]:
vector_size = 50  # output dim
window = 5        # context window
min_count = 5     # filter rare words
workers = multiprocessing.cpu_count() - 1

w2v_model = Word2Vec(
    sentences=all_sentences,
    vector_size=vector_size,
    window=window,
    min_count=min_count,
    workers=workers,
    sg=0
)

In [ ]:
# average vectors
def document_vector(word_list, model, vector_size):
    # Initialize a zero vector
    vector = np.zeros(vector_size)
    count = 0
    
    # Sum the vectors of all words in the text
    for word in word_list:
        if word in model.wv:
            vector += model.wv[word]
            count += 1
            
    # Return the average vector
    if count != 0:
        return vector / count
    else:
        # Return the zero vector if no words were found in the vocabulary
        return vector

### Prepare ngrams

In [101]:
titles = df['Title'].apply(lambda x: document_vector(tokenize_text(x), w2v_model, vector_size))
descriptions = df['FullDescription'].apply(lambda x: document_vector(tokenize_text(x), w2v_model, vector_size))

In [102]:
title_df = pd.DataFrame(titles.tolist(), index=df.index).add_prefix('Title_vec_')
desc_df = pd.DataFrame(descriptions.tolist(), index=df.index).add_prefix('FullDescription_vec_')

In [103]:
texts = pd.concat([title_df, desc_df], axis=1)

# Transformer

In [16]:
%pip install transformers -q
from transformers import RobertaTokenizer, RobertaModel
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm

Note: you may need to restart the kernel to use updated packages.


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [18]:
class RobertaFeatureDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

In [17]:
MAX_LEN = 128
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaModel.from_pretrained(model_name)
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cuda


In [61]:
# Prepare data

In [35]:
texts_concat = df['Title'] + ' [SEP] ' + df['FullDescription']
texts_list = texts_concat.tolist()

In [37]:
dataset = RobertaFeatureDataset(texts_list)
data_loader = DataLoader(dataset, batch_size=16, shuffle=False)

In [29]:
def extract_cls_vectors(model, data_loader, device):
    model.eval()
    all_cls_vectors = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Extraction [CLS] RoBERTa"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            cls_vector = outputs.last_hidden_state[:, 0, :]
            
            all_cls_vectors.append(cls_vector.cpu().numpy())

    final_vector_array = np.concatenate(all_cls_vectors, axis=0)
    return final_vector_array

In [40]:
final_vector_array = extract_cls_vectors(model, data_loader, device)

texts_w2v_roberta = pd.DataFrame(
    final_vector_array,
    index=df.index
)

texts_w2v_roberta.columns = [f'cls_{i}' for i in range(final_vector_array.shape[1])]

In [41]:
texts_w2v_roberta.head()

,cls_0,cls_1,cls_2,cls_3,cls_4,cls_5,cls_6,cls_7,cls_8,cls_9,cls_10,cls_11,cls_12,cls_13,cls_14,cls_15,cls_16,cls_17,cls_18,cls_19,cls_20,cls_21,cls_22,cls_23,cls_24,cls_25,cls_26,cls_27,cls_28,cls_29,cls_30,cls_31,cls_32,cls_33,cls_34,cls_35,cls_36,cls_37,cls_38,cls_39,cls_40,cls_41,cls_42,cls_43,cls_44,cls_45,cls_46,cls_47,cls_48,cls_49,cls_50,cls_51,cls_52,cls_53,cls_54,cls_55,cls_56,cls_57,cls_58,cls_59,cls_60,cls_61,cls_62,cls_63,cls_64,cls_65,cls_66,cls_67,cls_68,cls_69,cls_70,cls_71,cls_72,cls_73,cls_74,cls_75,cls_76,cls_77,cls_78,cls_79,cls_80,cls_81,cls_82,cls_83,cls_84,cls_85,cls_86,cls_87,cls_88,cls_89,cls_90,cls_91,cls_92,cls_93,cls_94,cls_95,cls_96,cls_97,cls_98,cls_99,cls_100,cls_101,cls_102,cls_103,cls_104,cls_105,cls_106,cls_107,cls_108,cls_109,cls_110,cls_111,cls_112,cls_113,cls_114,cls_115,cls_116,cls_117,cls_118,cls_119,cls_120,cls_121,cls_122,cls_123,cls_124,cls_125,cls_126,cls_127,cls_128,cls_129,cls_130,cls_131,cls_132,cls_133,cls_134,cls_135,cls_136,cls_137,cls_138,cls_139,cls_140,cls_141,cls_142,cls_143,cls_144,cls_145,cls_146,cls_147,cls_148,cls_149,cls_150,cls_151,cls_152,cls_153,cls_154,cls_155,cls_156,cls_157,cls_158,cls_159,cls_160,cls_161,cls_162,cls_163,cls_164,cls_165,cls_166,cls_167,cls_168,cls_169,cls_170,cls_171,cls_172,cls_173,cls_174,cls_175,cls_176,cls_177,cls_178,cls_179,cls_180,cls_181,cls_182,cls_183,cls_184,cls_185,cls_186,cls_187,cls_188,cls_189,cls_190,cls_191,cls_192,cls_193,cls_194,cls_195,cls_196,cls_197,cls_198,cls_199,cls_200,cls_201,cls_202,cls_203,cls_204,cls_205,cls_206,cls_207,cls_208,cls_209,cls_210,cls_211,cls_212,cls_213,cls_214,cls_215,cls_216,cls_217,cls_218,cls_219,cls_220,cls_221,cls_222,cls_223,cls_224,cls_225,cls_226,cls_227,cls_228,cls_229,cls_230,cls_231,cls_232,cls_233,cls_234,cls_235,cls_236,cls_237,cls_238,cls_239,cls_240,cls_241,cls_242,cls_243,cls_244,cls_245,cls_246,cls_247,cls_248,cls_249,cls_250,cls_251,cls_252,cls_253,cls_254,cls_255,cls_256,cls_257,cls_258,cls_259,cls_260,cls_261,cls_262,cls_263,cls_264,cls_265,cls_266,cls_267,cls_268,cls_269,cls_270,cls_271,cls_272,cls_273,cls_274,cls_275,cls_276,cls_277,cls_278,cls_279,cls_280,cls_281,cls_282,cls_283,cls_284,cls_285,cls_286,cls_287,cls_288,cls_289,cls_290,cls_291,cls_292,cls_293,cls_294,cls_295,cls_296,cls_297,cls_298,cls_299,cls_300,cls_301,cls_302,cls_303,cls_304,cls_305,cls_306,cls_307,cls_308,cls_309,cls_310,cls_311,cls_312,cls_313,cls_314,cls_315,cls_316,cls_317,cls_318,cls_319,cls_320,cls_321,cls_322,cls_323,cls_324,cls_325,cls_326,cls_327,cls_328,cls_329,cls_330,cls_331,cls_332,cls_333,cls_334,cls_335,cls_336,cls_337,cls_338,cls_339,cls_340,cls_341,cls_342,cls_343,cls_344,cls_345,cls_346,cls_347,cls_348,cls_349,cls_350,cls_351,cls_352,cls_353,cls_354,cls_355,cls_356,cls_357,cls_358,cls_359,cls_360,cls_361,cls_362,cls_363,cls_364,cls_365,cls_366,cls_367,cls_368,cls_369,cls_370,cls_371,cls_372,cls_373,cls_374,cls_375,cls_376,cls_377,cls_378,cls_379,cls_380,cls_381,cls_382,cls_383,cls_384,cls_385,cls_386,cls_387,cls_388,cls_389,cls_390,cls_391,cls_392,cls_393,cls_394,cls_395,cls_396,cls_397,cls_398,cls_399,cls_400,cls_401,cls_402,cls_403,cls_404,cls_405,cls_406,cls_407,cls_408,cls_409,cls_410,cls_411,cls_412,cls_413,cls_414,cls_415,cls_416,cls_417,cls_418,cls_419,cls_420,cls_421,cls_422,cls_423,cls_424,cls_425,cls_426,cls_427,cls_428,cls_429,cls_430,cls_431,cls_432,cls_433,cls_434,cls_435,cls_436,cls_437,cls_438,cls_439,cls_440,cls_441,cls_442,cls_443,cls_444,cls_445,cls_446,cls_447,cls_448,cls_449,cls_450,cls_451,cls_452,cls_453,cls_454,cls_455,cls_456,cls_457,cls_458,cls_459,cls_460,cls_461,cls_462,cls_463,cls_464,cls_465,cls_466,cls_467,cls_468,cls_469,cls_470,cls_471,cls_472,cls_473,cls_474,cls_475,cls_476,cls_477,cls_478,cls_479,cls_480,cls_481,cls_482,cls_483,cls_484,cls_485,cls_486,cls_487,cls_488,cls_489,cls_490,cls_491,cls_492,cls_493,cls_494,cls_495,cls_496,cls_497,cls_498,cls_499,cls_500,cls_501,cls_502,cls_503,cls_504,cls_505,cls_506,cls_507,cls_508,cls_509,cls_510,cls_511,cls_512,cls_5

# title-uni_gram & description-roberta

roberta

In [ ]:
%pip install transformers -q
from transformers import RobertaTokenizer, RobertaModel
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm

In [ ]:
MAX_LEN = 128
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaModel.from_pretrained(model_name)
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

In [19]:
description_ds = RobertaFeatureDataset(df['FullDescription'].tolist())
description_dl = DataLoader(description_ds, batch_size=16, shuffle=False)

In [30]:
final_vector_array = extract_cls_vectors(model, description_dl, device)

description_roberta = pd.DataFrame(
    final_vector_array,
    index=df.index
)

description_roberta.columns = [f'cls_{i}' for i in range(final_vector_array.shape[1])]

Extraction [CLS] RoBERTa: 100%|██████████| 15298/15298 [31:13<00:00,  8.16it/s]


In [31]:
desc_roberta_train = description_roberta.loc[train.index]
desc_roberta_test  = description_roberta.loc[test.index]

tf-idf

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.decomposition import TruncatedSVD
import joblib

In [41]:
tfidf_title = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 1))
X_train_title = tfidf_title.fit_transform(train["Title"])
X_test_title  = tfidf_title.transform(test["Title"])

In [42]:
from sklearn.utils.extmath import randomized_svd

U, Sigma, VT = randomized_svd(X_train_title, n_components=50, n_iter=5, random_state=42)
X_train_title = U @ np.diag(Sigma)
X_test_title  = X_test_title.dot(VT.T)

In [ ]:
# svd = TruncatedSVD(n_components=50, random_state=42, verbose=1)

# X_train_text = svd.fit_transform(X_train_title)
# X_test_text  = svd.transform(X_test_title)

In [46]:
X_train_title.shape

(171337, 50)

In [47]:
desc_roberta_train.shape

(171337, 768)

In [38]:
df.shape

(244768, 9)

In [39]:
train.shape

(171337, 9)

In [49]:
train_text_combined = np.concatenate([desc_roberta_train, X_train_title], axis=1)
test_text_combined = np.concatenate([desc_roberta_test, X_test_title], axis=1)

In [ ]:
train_text_combined_df = pd.DataFrame(train_text_combined)
test_text_combined_df = pd.DataFrame(test_text_combined)

train_text_combined_df.to_parquet('data/texts_uni_roberta_train.parquet', index=True)
test_text_combined_df.to_parquet('data/texts_uni_roberta_test.parquet',  index=True)

In [62]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs        7.7G     0  7.7G   0% /dev
tmpfs           7.7G  4.0K  7.7G   1% /dev/shm
tmpfs           7.7G  672K  7.7G   1% /run
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  135G   78G   58G  58% /
tmpfs           1.6G     0  1.6G   0% /run/user/0
/dev/nvme2n1    4.8G  4.4G  146M  97% /home/ec2-user/SageMaker
tmpfs           1.6G     0  1.6G   0% /run/user/1001
tmpfs           1.6G     0  1.6G   0% /run/user/1000
tmpfs           1.6G     0  1.6G   0% /run/user/1002


In [57]:
test_text_combined_df.to_parquet('data/texts_uni_roberta_test.parquet',  index=True)

OSError: [Errno 28] Error writing bytes to file. Detail: [errno 28] No space left on device

# 5. Split data

In [25]:
# tabular
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3, random_state=42)

In [106]:
# word2vec
texts_train = texts.loc[train.index]
texts_test = texts.loc[test.index]
texts_train.to_pickle('data/texts_w2v_train.pkl')
texts_test.to_pickle('data/texts_w2v_test.pkl')

In [44]:
# roberta
texts_roberta_train = texts_w2v_roberta.loc[train.index]
texts_roberta_test  = texts_w2v_roberta.loc[test.index]
texts_roberta_train.to_parquet('data/texts_roberta_train.parquet', index=True)
texts_roberta_test.to_parquet('data/texts_roberta_test.parquet',  index=True)

# 6. One hot encoding

In [57]:
# select most common source in category group
category_to_source = train.groupby('Category')['SourceName'].agg(lambda x: x.mode()[0]).to_dict()
train['SourceName'] = train['Category'].map(category_to_source)
test['SourceName'] = test['Category'].map(category_to_source)

In [58]:
train = pd.get_dummies(train, columns = ['ContractType', 'ContractTime', 'Category', 'SourceName'], drop_first=True, dtype=int)
test = pd.get_dummies(test, columns = ['ContractType', 'ContractTime', 'Category', 'SourceName'], drop_first=True, dtype=int)

# 7. Target Encoding - mean salary of company instead of company name

In [59]:
# combining companies by two first words
train['CompanyPrefix'] = train['Company'].apply(lambda x: ' '.join(str(x).split()[:2]))
test['CompanyPrefix'] = test['Company'].apply(lambda x: ' '.join(str(x).split()[:2]))

In [60]:
# mean salary by company
mean_company = train.groupby('CompanyPrefix')['SalaryNormalized'].mean()
train['CompanyEncoded'] = train['CompanyPrefix'].map(mean_company)
test['CompanyEncoded'] = test['CompanyPrefix'].map(mean_company)

# filling not existing companies in test with global mean
global_mean = train['SalaryNormalized'].mean()
test['CompanyEncoded'] = test['CompanyEncoded'].fillna(global_mean)

train.drop(columns=['Company', 'CompanyPrefix'], inplace=True)
test.drop(columns=['Company', 'CompanyPrefix'], inplace=True)

In [107]:
mean_company.head()

CompanyPrefix
.Michael Page    77500.0
1 1              21040.0
10 TRINITY       45000.0
100 percent      40500.0
100% IT          30250.0
Name: SalaryNormalized, dtype: float64

# 8. Standarization

In [62]:
from sklearn.preprocessing import StandardScaler
numeric_cols = ['SalaryNormalized', 'CompanyEncoded', 'LocationPopulation']
scaler = StandardScaler()
train[numeric_cols] = scaler.fit_transform(train[numeric_cols])
test[numeric_cols] = scaler.transform(test[numeric_cols])

# 9. Tf-idf

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.decomposition import TruncatedSVD
import joblib

In [22]:
def prepare_tfidf(train, test, n_grams):
    tfidf_description = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=n_grams)
    tfidf_title = TfidfVectorizer(max_features=1000, stop_words='english', ngram_range=n_grams)
    
    X_train_description = tfidf_description.fit_transform(train["FullDescription"])
    X_test_description  = tfidf_description.transform(test["FullDescription"])
    
    X_train_title = tfidf_title.fit_transform(train["Title"])
    X_test_title  = tfidf_title.transform(test["Title"])
    
    return hstack([X_train_description, X_train_title]), hstack([X_test_description, X_test_title])

In [ ]:
X_train_text, X_test_text = prepare_tfidf(train, test, (1, 3))

In [ ]:
# dimenshion reduction
svd = TruncatedSVD(n_components=50, random_state=42)

X_train_text = svd.fit_transform(X_train_text)
X_test_text  = svd.transform(X_test_text)

In [ ]:
# saving
np.save("data/X_train_text_tri.npy", X_train_text)
np.save("data/X_test_text_tri.npy", X_test_text)

# 10. Tabular data saving

In [46]:
train_tab = train.drop(columns=['Title', 'FullDescription'])
test_tab = test.drop(columns=['Title', 'FullDescription'])

In [47]:
train_tab.to_csv('data/train_preprocessed.csv', index=False)
test_tab.to_csv('data/test_preprocessed.csv', index=False)

In [51]:
train_tab.head(3)

,SalaryNormalized,LocationPopulation,ContractType_full_time,ContractType_part_time,ContractTime_contract,ContractTime_permanent,Category_Admin Jobs,Category_Charity & Voluntary Jobs,Category_Consultancy Jobs,Category_Creative & Design Jobs,Category_Customer Services Jobs,Category_Domestic help & Cleaning Jobs,"Category_Energy, Oil & Gas Jobs",Category_Engineering Jobs,Category_Graduate Jobs,Category_HR & Recruitment Jobs,Category_Healthcare & Nursing Jobs,Category_Hospitality & Catering Jobs,Category_IT Jobs,Category_Legal Jobs,Category_Logistics & Warehouse Jobs,Category_Maintenance Jobs,Category_Manufacturing Jobs,Category_Other/General Jobs,"Category_PR, Advertising & Marketing Jobs",Category_Part time Jobs,Category_Property Jobs,Category_Retail Jobs,Category_Sales Jobs,Category_Scientific & QA Jobs,Category_Social work Jobs,Category_Teaching Jobs,Category_Trade & Construction Jobs,Category_Travel Jobs,SourceName_Jobcentre Plus,SourceName_TotallyLegal,SourceName_caterer.com,SourceName_charityjob.co.uk,SourceName_cv-library.co.uk,SourceName_cwjobs.co.uk,SourceName_findababysitter.com,SourceName_jobs.communitycare.co.uk,SourceName_retailchoice.com,SourceName_staffnurse.com,SourceName_totaljobs.com,CompanyEncoded
241453,-0.857472,1.095796,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1.474048
169520,-0.281558,1.095796,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,-0.903872
136661,-0.631001,1.095796,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-0.706080
